<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


# DAY1

### Learning Objectives
- Describe the foundational components of a neural network
- Implement a Perceptron from scratch in Python

#### Input Layer:

The input Layer is where the feature data from the dataframe are input or where inputs from other neurons are recieved.

#### Hidden Layer:

These are the layer that exist between the input layer and output layer. You cna have one hidden layer or many hidden layers

#### Output Layer:

This is the answer/result of our neurons in our neural netoworks. These ouputs can then be used as inputs for the next layer of neurons or be the final output(s) of the neural network.

#### Neuron:

The neuron recieves inputs, multiplies the inputs by their weights, sums everyhting up, and then applies the activation function to the sum. Usually involves a continuous activation function

#### Weight:

This is the amount or positive or negative effect an input will be associated with the ending output.

#### Activation Function:

The activation function is how the neural network normalizes the results after inputs, weights, and biases have been applied within the neuron.

#### Node Map:

The node maps show how the features of the dataframe or the outputs of upper level neurons are further processed throughout the neural netowork. It shows inputs, outputs, and hidden layers visualized at a high level.

#### Perceptron:

Simply, a perceptron consists of four distinct parts. Uses a binary activation function that is either activate or not, different from a neuron

    Inputs
    Weights
    Weighted Sum
    Activation Function (Output)

Perceptrons classify data into two parts (0,1) most of the time. Perceptrons are also known as Linear Binary Classifiers


#### Inputs -> Outputs
Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?
Your Answer Here

Depending on your network, Inputs and Outputs can range arbitraily. Each input can come from an upper level neuron or the intial inputted values from a dataframe. Each input can be weighted negatively or positvely depending on whether your desired answer needs the neuron to activate negatively or positively depending how your inputted bias has shifted the activation curve up or down.


### Imports

In [1]:
!pip install category-encoders

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import category_encoders as ce

#### Clean/Load Data

In [286]:
#Load Data
df = sns.load_dataset('tips')
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [292]:
df['bill_tip_sum'] = pd.qcut(df['total_bill']+df['tip'], 3, labels=['low', 'medium', 'high'])

In [293]:
print(df.shape)
df.head()

(244, 8)


,total_bill,tip,sex,smoker,day,time,size,bill_tip_sum
0,16.99,1.01,Female,No,Sun,Dinner,2,medium
1,10.34,1.66,Male,No,Sun,Dinner,3,low
2,21.01,3.50,Male,No,Sun,Dinner,3,medium
3,23.68,3.31,Male,No,Sun,Dinner,2,high
4,24.59,3.61,Female,No,Sun,Dinner,4,high


In [294]:
def prep(df, target):
    
    """
    This function will:
    1. Change "size" into a catagorical to be one hotted
    2. Add Total and Tip and put into 3 bins
    3. Split data
    4. Create X and y train/test
    5. process X train/test data by one hotting categoricals
    6. Make 'sex' a binary column
    7. return 4 df's
    """
    df['size'] = df['size'].astype(str)
    df['bill_tip_sum'] = pd.qcut(df['total_bill']+df['tip'], 3, labels=['low', 'medium', 'high'])
    
    training, testing = train_test_split(df, test_size=.2)
    
    X_train = training.drop(columns=target)
    y_train = training[target]
    X_test = testing.drop(columns=target)
    y_test = testing[target]
    
    processor = make_pipeline(
        ce.OneHotEncoder(use_cat_names=True),  
 #       SimpleImputer(strategy='median'),
 #       StandardScaler()
    )
    
    gender = {'Female': 0, 'Male': 1}
    y_train = y_train.map(gender)
    y_test = y_test.map(gender)
    
    X_process_train = processor.fit_transform(X_train)
    X_process_test = processor.transform(X_test)
    
    return X_process_train,y_train, X_process_test, y_test

In [296]:
X_train, y_train, X_test, y_test = prep(df, 'sex')
print(X_train.shape) 
print(X_test.shape) 
print(y_train.shape) 
print(y_test.shape)
X_train.head()

(195, 19)
(49, 19)
(195,)
(49,)


,total_bill,tip,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner,size_2,size_5,size_4,size_1,size_6,size_3,bill_tip_sum_low,bill_tip_sum_medium,bill_tip_sum_high
94,22.75,3.25,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1
155,29.85,5.14,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1
144,16.43,2.30,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0
4,24.59,3.61,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1
7,26.88,3.12,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1


#### Single Layer Perceptron

In [338]:
class NNet:
    def __init__(self):
        
        # Inputs must be == to number of features
        self.inputs = 19
        # Only one output node b/c only trying to predict one thing
        self.outputNodes = 1
        
        self.weights = np.random.rand(self.inputs, self.outputNodes)
     
    # Squishify
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    # Create 0 or 1 from prediced activated output
    def binary(self, X):
        binary = self.feed_forward(X)
        binary = [1 if x > .99999 else 0 for x in binary]
        return binary
    
     
    def feed_forward(self, X):
        """Calculate the NNet inference using the feed forward, aka predict """
        
        # Combining  inputs and weights in a weighted sum
        self.input_sum = np.dot(X, self.weights)
        
        # Apply activation function to the weighted sum
        self.output_activated = self.sigmoid(self.input_sum)
        
        return self.output_activated

In [339]:
nn = NNet()
y_pred1 = nn.binary(X_train)
score = accuracy_score(y_train, y_pred1)
score

0.5128205128205128

In [340]:
y_pred2 = nn.binary(X_test)
score = accuracy_score(y_test, y_pred2)
score

0.42857142857142855

In [341]:
preddy = nn.feed_forward(X_train)

nn.feed_forward(X_train)

array([[0.99999336],
       [0.99999989],
       [0.99984708],
       [0.99999701],
       [0.99999759],
       [0.99672527],
       [0.99819794],
       [0.99999978],
       [0.99997168],
       [0.99999968],
       [0.99999998],
       [0.99998752],
       [0.99999741],
       [0.99999879],
       [0.99999697],
       [0.99996832],
       [0.99954236],
       [0.99993134],
       [0.99999935],
       [0.99996606],
       [0.99998643],
       [0.99997716],
       [0.99901165],
       [0.99990891],
       [0.99998085],
       [0.99999774],
       [0.99999761],
       [0.99916846],
       [0.99996913],
       [0.99992043],
       [0.99983532],
       [0.99964271],
       [0.99983398],
       [0.99999828],
       [0.99878582],
       [0.99855479],
       [0.99996721],
       [0.99999952],
       [0.99998637],
       [0.99995424],
       [0.99976118],
       [0.99965541],
       [0.99999247],
       [0.99996839],
       [0.99988967],
       [0.99999813],
       [0.9999535 ],
       [0.999

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?